# rhino3dm exercise 02

## import and util funcs

In [1]:
import rhino3dm
from dataclasses import dataclass, field
import random
import string

print('rhino3dm ver: ', rhino3dm.__version__)

def generate_id() -> str:
    return "".join(random.choices(string.ascii_lowercase, k=12))

rhino3dm ver:  8.6.0

## read `.3dm` file

In [2]:
rhino_f_path = "./data/nongke.3dm"
file3dm_obj = rhino3dm.File3dm.Read(rhino_f_path)
print('rhino3dm.File3dm object: ', file3dm_obj)

rhino3dm.File3dm object:  <rhino3dm._rhino3dm.File3dm object at 0x000001661ABD0DF0>

## read `notes`

In [3]:
file3dm_notes = rhino3dm.File3dm.ReadNotes(rhino_f_path).split(';')[:-1]

for idx, note in enumerate(file3dm_notes):
    print(
        note.strip().split(':')[0].strip(),
        ' : ',
        note.strip().split(':')[1].strip()
    )

name  :  nongke
lati  :  20
long  :  100

> **File3dm.ReadNotes()**
>
> [***method***
> `rhino3dm.File3dm.ReadNotes()`](https://mcneel.github.io/rhino3dm/python/api/File3dm.html#rhino3dm.File3dm.ReadNotes)
>
> <br />
>
> Reads only the notes from an existing 3dm file.
>
> <br />
>
> Parameters:
>
> -   path (str) – The file from which to read the notes.
>
> Returns:
>
> -   The 3dm file notes.
>
> Return type:
>
> -   str

## read `Layer`

In [4]:
layer_table = file3dm_obj.Layers
print('layer table object: ', layer_table)
print('num of layers: ', len(layer_table))
print()

for idx, layer in enumerate(iter(layer_table)):
    print('layer ', idx, ' Object  : ', layer)
    print('      ', idx, ' Id      : ', layer.Id)
    print('      ', idx, ' Index   : ', layer.Index)
    print('      ', idx, ' Name    : ', layer.Name)
    print('      ', idx, ' Fullpath: ', layer.FullPath)
    print('      ', idx, ' Color   : ', layer.Color)
    print()

layer table object:  <rhino3dm._rhino3dm.File3dmLayerTable object at 0x000001661B0D6630>
num of layers:  6

layer  0  Object  :  <rhino3dm._rhino3dm.Layer object at 0x000001661B0D6B30>
       0  Id      :  212960d6-ad46-439d-8516-772ce85fde03
       0  Index   :  0
       0  Name    :  0
       0  Fullpath:  0
       0  Color   :  (0, 0, 0, 255)

layer  1  Object  :  <rhino3dm._rhino3dm.Layer object at 0x000001661B0D77B0>
       1  Id      :  25488de8-55a5-470b-a098-ea3c8c1220b3
       1  Index   :  1
       1  Name    :  cable
       1  Fullpath:  cable
       1  Color   :  (255, 191, 0, 255)

layer  2  Object  :  <rhino3dm._rhino3dm.Layer object at 0x000001661B0D45F0>
       2  Id      :  33892670-4a65-4460-9e7e-78d057c02742
       2  Index   :  2
       2  Name    :  pt
       2  Fullpath:  pt
       2  Color   :  (0, 0, 0, 255)

layer  3  Object  :  <rhino3dm._rhino3dm.Layer object at 0x000001661B0D4F70>
       3  Id      :  c0c0bbb7-2adb-4301-8c42-c09459068132
       3  Index   : 

## read `ObjectsTable` in the `File3dm` object

In [5]:
object_table = file3dm_obj.Objects
print('object table: ', object_table, '\n')

object table:  <rhino3dm._rhino3dm.File3dmObjectTable object at 0x000001661B0D6F70> 


## create `OatPV` class

In [6]:
@dataclass
class OatPV:
    idx: int
    outline: rhino3dm.PolylineCurve
    str_num: str = ''
    id: str = field(default_factory=generate_id)

## read `Geometry` on the `pv` layer

In [7]:
layer_idx_of_pv = 3
cnt_pv = 0
max_num_obj = 3
total_num_pv = 0

oatPV_list = []

for idx, obj in enumerate(iter(object_table)):
    # print the first 3 obj on a layer
    if (obj.Attributes.LayerIndex == layer_idx_of_pv) and (cnt_pv < max_num_obj):
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()

        # if geom is PolylineCurve, print its details
        if isinstance(obj.Geometry, rhino3dm.PolylineCurve):
            print('            curve IsClosed     :', obj.Geometry.IsClosed)
            print('            curve IsPlanar     :', obj.Geometry.IsPlanar())
            print('            curve IsPolyline   :', obj.Geometry.IsPolyline())
            print()

            print('            curve PointCount   :', obj.Geometry.PointCount)
            
            for i in range(obj.Geometry.PointCount):
                print('            curve pt ', i, ' : ', obj.Geometry.Point(i))
                # print('           curve pt ', i, ' : ', obj.Geometry.ToPolyline().PointAt(i)) -- alternative

            print()

            print('            curve SegmentCount :', obj.Geometry.ToPolyline().SegmentCount)

            for i in range(obj.Geometry.ToPolyline().SegmentCount):
                print('            segement ', i, ' : ', obj.Geometry.ToPolyline().SegmentAt(i))

            print()

            oatPV = OatPV(idx, obj.Geometry)
            oatPV_list.append(oatPV)

        cnt_pv += 1
    else:
        pass

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pv):
        total_num_pv += 1

print('total num of pv: ', total_num_pv)
print()

for idx, oatPV in enumerate(oatPV_list):
    print('oatPV ', idx, ' : ', oatPV)

object  2547  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x000001661AC0E1F0>
        2547  Geometry              :  <rhino3dm._rhino3dm.Brep object at 0x000001661B0B54B0>
        2547  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000001661B0B54B0>
        2547  Attributes.Id         :  9f7de04f-c267-4afb-8834-cd6dd24f42a0
        2547  Attributes.Name       :  
        2547  Attributes.LayerIndex :  3
        2547  Attributes.ObjectColor:  (0, 0, 0, 255)
        2547  Attributes.Visible    :  True

object  2548  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x00000166712E49F0>
        2548  Geometry              :  <rhino3dm._rhino3dm.Brep object at 0x000001661AC0E1F0>
        2548  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000001661AC0E1F0>
        2548  Attributes.Id         :  ad6b16b1-4afc-47b5-af38-7ae30d9f379b
        2548  Attributes.Name       :  
        2548  Attribut

## read `Geometry` on the `cable` layer

In [8]:
layer_idx_of_cable = 1
cnt_cable = 0
max_num_obj = 3
total_num_cable = 0

for idx, obj in enumerate(iter(object_table)):
    # print the first 3 obj on a layer
    if (obj.Attributes.LayerIndex == layer_idx_of_cable) and (cnt_cable < max_num_obj):
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()

        # if geom is PolylineCurve, print its details
        if isinstance(obj.Geometry, rhino3dm.PolylineCurve):
            print('            curve IsClosed     :', obj.Geometry.IsClosed)
            print('            curve IsPlanar     :', obj.Geometry.IsPlanar())
            print('            curve IsPolyline   :', obj.Geometry.IsPolyline())
            print()

            print('            curve PointCount   :', obj.Geometry.PointCount)
            
            for i in range(obj.Geometry.PointCount):
                print('            curve pt ', i, ' : ', obj.Geometry.Point(i))
                # print('           curve pt ', i, ' : ', obj.Geometry.ToPolyline().PointAt(i)) -- alternative

            print()

            print('            curve SegmentCount :', obj.Geometry.ToPolyline().SegmentCount)

            for i in range(obj.Geometry.ToPolyline().SegmentCount):
                print('            segement ', i, ' : ', obj.Geometry.ToPolyline().SegmentAt(i))

            print()

        cnt_cable += 1
    else:
        pass

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_cable):
        total_num_cable += 1

print('total num of cable: ', total_num_cable)

object  0  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x000001661B0B5F30>
        0  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x000001661ABC8B30>
        0  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000001661ABC8B30>
        0  Attributes.Id         :  fa8bcce9-953e-47f0-bcfc-4de7a87e0c76
        0  Attributes.Name       :  
        0  Attributes.LayerIndex :  1
        0  Attributes.ObjectColor:  (0, 0, 0, 255)
        0  Attributes.Visible    :  True

            curve IsClosed     : False
            curve IsPlanar     : True
            curve IsPolyline   : True

            curve PointCount   : 6
            curve pt  0  :  14.298569362095407,251.0734261769187,0.0
            curve pt  1  :  2.232971074135598,253.20091370662107,0.0
            curve pt  2  :  1.9377067622771804,251.5263865827965,0.0
            curve pt  3  :  14.003304539454732,249.398896156304,0.0
            curve pt  4  :  13

## read `Text` on `text` layer

In [9]:
layer_idx_of_text = 4
cnt_text = 0
max_num_obj = 3
total_num_text = 0

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_text) and (cnt_text < max_num_obj):
        print('object ', idx, ' Object                  : ', obj)
        print('       ', idx, ' Geometry                : ', obj.Geometry)
        print('       ', idx, ' Attributes              : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id           : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name         : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex   : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor  : ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible      : ', obj.Attributes.Visible)
        print()

        # if geom is AnnotationBase, print its PlainText
        if isinstance(obj.Geometry, rhino3dm.AnnotationBase):
            print('            Geometry.PlainText     : ', obj.Geometry.PlainText)
            print()

        cnt_text += 1

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_text):
        total_num_text += 1

print('total num of text: ', total_num_text)

object  72  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000001661ABD9EF0>
        72  Geometry                :  <rhino3dm._rhino3dm.PolylineCurve object at 0x000001661ABD1570>
        72  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000001661ABD1570>
        72  Attributes.Id           :  36f08c55-5986-4aa2-b0b8-ae56b26b043c
        72  Attributes.Name         :  
        72  Attributes.LayerIndex   :  4
        72  Attributes.ObjectColor  :  (0, 0, 0, 255)
        72  Attributes.Visible      :  True

object  73  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000001661B0DE970>
        73  Geometry                :  <rhino3dm._rhino3dm.PolylineCurve object at 0x000001661ABD9EF0>
        73  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000001661ABD9EF0>
        73  Attributes.Id           :  2a5e0e48-e0ca-4d27-896e-1d3f2435645e
        73  Attributes.Name         :  
    

## create a dataclass `MyPt`

In [10]:
@dataclass
class OatPt:
    idx  : int
    point: rhino3dm.Point3d
    name : str

## read `Point` on `pt` layer

In [11]:
layer_idx_of_pt = 2
cnt_pt = 0
max_num_obj = 3
total_num_pt = 0

oaPt_list_1 = []

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pt) and (cnt_pt < max_num_obj):
        print('object ', idx, ' Object                  : ', obj)
        print('       ', idx, ' Geometry                : ', obj.Geometry)
        print('       ', idx, ' Attributes              : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id           : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name         : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex   : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor  : ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible      : ', obj.Attributes.Visible)
        print()

        # if geom is Point, print its xyz
        if isinstance(obj.Geometry, rhino3dm.Point):
            pt = obj.Geometry.Location
            print('            Geometry.Location.X     : ', pt.X)
            print('            Geometry.Location.Y     : ', pt.Y)
            print('            Geometry.Location.Z     : ', pt.Z)

            oatPt = OatPt(idx, pt, str(idx))
            oaPt_list_1.append(oatPt)

            print()

for idx, obj in enumerate(iter(object_table)):
    if (obj.Attributes.LayerIndex == layer_idx_of_pt):
        total_num_pt += 1

print('total num of pt: ', total_num_pt)

object  2544  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000001661AC25030>
        2544  Geometry                :  <rhino3dm._rhino3dm.Point object at 0x000001661ABD09B0>
        2544  Attributes              :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000001661ABD09B0>
        2544  Attributes.Id           :  4a631072-56bf-4491-8ff5-0a7e5696c4f7
        2544  Attributes.Name         :  
        2544  Attributes.LayerIndex   :  2
        2544  Attributes.ObjectColor  :  (0, 0, 0, 255)
        2544  Attributes.Visible      :  True

            Geometry.Location.X     :  2.206781230039775
            Geometry.Location.Y     :  251.68157067516623
            Geometry.Location.Z     :  0.0

object  2545  Object                  :  <rhino3dm._rhino3dm.File3dmObject object at 0x000001667123F5F0>
        2545  Geometry                :  <rhino3dm._rhino3dm.Point object at 0x000001661AC25030>
        2545  Attributes              :  <rhino3dm._rhino3dm.Ob

In [12]:
for idx, oatPt in enumerate(oaPt_list_1):
    print('myPt', idx, ' : ', oatPt)
    print('    type of idx in myPt:', type(oatPt.idx))
    print('    type of point in myPt:', type(oatPt.point))
    print('    type of name in myPt:', type(oatPt.name))

myPt 0  :  OatPt(idx=2544, point=Point3d(2.20678, 251.682, 0), name='2544')
    type of idx in myPt: <class 'int'>
    type of point in myPt: <class 'rhino3dm._rhino3dm.Point3d'>
    type of name in myPt: <class 'str'>
myPt 1  :  OatPt(idx=2545, point=Point3d(1.57609, 248.355, 0), name='2545')
    type of idx in myPt: <class 'int'>
    type of point in myPt: <class 'rhino3dm._rhino3dm.Point3d'>
    type of name in myPt: <class 'str'>
myPt 2  :  OatPt(idx=2546, point=Point3d(15.9975, 247.462, 0), name='2546')
    type of idx in myPt: <class 'int'>
    type of point in myPt: <class 'rhino3dm._rhino3dm.Point3d'>
    type of name in myPt: <class 'str'>

aaa